# IATI Currency Converion with Pandas

In [51]:
import pandas as pd
import numpy as np

In [52]:
# Exchange Rate Data

exchange_df = pd.read_csv("../../data/rates_06_07_2022.csv", dtype={"Rate": np.float64 }, parse_dates=["Date"])

# example accessing a row of data
exchange_df[(exchange_df["Date"] == pd.Timestamp('2022-01-01')) & (exchange_df["Currency"] == 'EUR')]

,Date,Currency,Rate
145609,2022-01-01,EUR,1.134682


In [53]:
# Currency Conversion Function
from cmath import isnan


def convert_currency(date, currency, value):
    rates = exchange_df[(exchange_df["Date"] == date) & (exchange_df["Currency"] == currency)]
    if len(rates) > 0:
        exchange_rate = exchange_df.at[rates.index[0], "Rate"]
        if exchange_rate:
            return exchange_rate * value
    return np.NaN

# Mean Currency Conversion Function
def convert_currency_mean(start_date, end_date, currency, value):
    rates = exchange_df[((exchange_df["Date"] >= start_date) & (exchange_df["Date"] <= end_date)) & (exchange_df["Currency"] == currency)]
    if len(rates) > 0:
        exchange_rate = np.mean(a=rates["Rate"])
        if exchange_rate:
            return exchange_rate * value
    print(f"No mean exhange rate found for {currency} from {start_date} to {end_date}")
    return np.NaN

# function to get only the date part of the date, not the time
def conv_date(date):
    if pd.isnull(date):
        return ""
    return date.split('T')[0]

In [54]:
# Transaction Currency Conversion

transaction_df = pd.read_csv("../../data/sample_transactions.csv", dtype={"transaction_value": np.float64})

# get column with currency to use for conversion, first choice is transaction_value_currency, with default_currency fallback
transaction_df["currency_to_use"] = transaction_df["transaction_value_currency"].combine_first(transaction_df["default_currency"])

# get column with date to use for conversion, first choice is transaction_value_value_date, with transaction_transaction_date_iso_date fallback
transaction_df["date_to_use"] = transaction_df["transaction_value_value_date"].combine_first(transaction_df["transaction_transaction_date_iso_date"])

# get only the date part of the date, not the time
transaction_df["date_to_use"] = pd.to_datetime(transaction_df["date_to_use"].apply(lambda x: conv_date(x)))

transaction_df['transaction_value_USD'] = transaction_df.apply(lambda x: convert_currency(date=x["date_to_use"],currency=x["currency_to_use"], value = x["transaction_value"]), axis=1)

transaction_df[["currency_to_use", "date_to_use", "transaction_value", "transaction_value_USD"]]

,currency_to_use,date_to_use,transaction_value,transaction_value_USD
0,DKK,2021-11-10,200000.00,1.287000e+06
1,GBP,2020-10-26,510561.88,6.665385e+05
2,GBP,2020-10-01,104965.64,1.347391e+05
3,EUR,2020-12-31,8045.00,9.857270e+03
4,EUR,2015-01-01,210.00,2.540118e+02
5,EUR,2015-12-31,210.00,2.286270e+02
6,USD,2022-01-01,249178.00,2.491780e+05
7,USD,2019-01-01,541059.00,5.410590e+05
8,USD,2020-01-01,8118514.00,8.118514e+06
9,USD,2021-01-01,3599264.00,3.599264e+06


In [55]:
# Transaction Tests

assert transaction_df["transaction_value"].iloc[2] == 104965.64, "transaction_value 3rd row should be 104965.64"
assert transaction_df["transaction_value_USD"].iloc[2] == 104965.64 * 1.28365 , "transaction_value_USD 3rd row should be 104965.64 * 1.28365"

assert transaction_df["transaction_value"].iloc[6] == transaction_df["transaction_value_USD"].iloc[6], "transaction_value for USD row 7 should equal transaction_value_USD"

In [56]:
# Budget Currency Conversion

budget_df = pd.read_csv("../../data/sample_budgets.csv", dtype={"budget_value": np.float64}, converters={ "budget_value_value_date": conv_date, "budget_period_start_iso_date": conv_date, "budget_period_end_iso_date": conv_date })

# get column with currency to use for conversion, first choice is budget_value_currency, with default_currency fallback
budget_df["currency_to_use"] = budget_df["budget_value_currency"].combine_first(budget_df["default_currency"])

# convert date columns to dates
for col in ["budget_value_value_date", "budget_period_start_iso_date", "budget_period_end_iso_date"]:
    budget_df[col] = pd.to_datetime(budget_df[col])

# convert currency
def convert_budget_currency(row):
    [budget_value, currency_to_use, budget_value_value_date, budget_period_start_iso_date, budget_period_end_iso_date] = row
    # Try conversion with budget value date first, as is canonical conversion date
    if budget_value != "" and currency_to_use != "" and budget_value_value_date != "":
        return convert_currency(row["budget_value_value_date"], row["currency_to_use"], row["budget_value"])
    # Failing to find budget value date, use the period average for budget period
    elif budget_value != "" and currency_to_use != "" and budget_period_start_iso_date != "" and budget_period_end_iso_date != "":
        return convert_currency_mean(budget_period_start_iso_date, budget_period_end_iso_date,currency_to_use, budget_value )

budget_df['budget_value_USD'] = budget_df.apply(lambda x: convert_budget_currency(x[["budget_value", "currency_to_use", "budget_value_value_date", "budget_period_start_iso_date", "budget_period_end_iso_date"]]), axis=1)

budget_df[["budget_value_value_date","budget_period_start_iso_date", "budget_period_end_iso_date", "currency_to_use", "budget_value", "budget_value_USD"]].head(10)


,budget_value_value_date,budget_period_start_iso_date,budget_period_end_iso_date,currency_to_use,budget_value,budget_value_USD
0,2022-07-25,2012-09-03,2015-01-02,EUR,0.0,NaN
1,2022-07-25,2016-04-18,2017-04-17,EUR,0.0,NaN
2,2022-07-25,2022-01-01,2022-12-31,EUR,0.0,NaN
3,2018-03-16,2018-03-16,2018-06-30,USD,40000000.0,4.000000e+07
4,2022-01-01,2014-01-01,2014-12-31,EUR,14567.0,1.652891e+04
5,2022-01-01,2016-01-01,2016-12-31,EUR,114701.0,1.301491e+05
6,2022-01-01,2017-01-01,2017-03-31,EUR,0.0,0.000000e+00
7,2022-01-01,2017-04-01,2017-06-30,EUR,48388.0,5.490498e+04
8,NaT,NaT,NaT,MXN,NaN,NaN
9,2022-07-25,2012-02-01,2012-06-02,EUR,0.0,NaN


In [57]:
# Budget Tests

assert budget_df["budget_value"].iloc[4] == 14567.0, "budget_df 5th row should be 14567.0"
assert budget_df["budget_value_USD"].iloc[4] == 16528.91004545457, "budget_df 5th row should be 16528.91004545457"

assert budget_df["budget_value"].iloc[3] == budget_df["budget_value_USD"].iloc[3], "budget_value for USD row 4 should equal budget_value_USD"